# SQL Section

In Figure 1 in the Appendix, you can see a depiction of the database described in the prompt
above. It may be valuable to answering questions below. If you get stumped, please write out
your thoughts. <br>
<br>
    <li> We have decided to expand our data model by adding a new Case Type table. Each case
in Mighty has exactly one “Case Type” (such as Motor Vehicle Accident or Other).
Assuming the only information on the Case Type table is the primary key and the name
of the case type. Please add this table to Figure 1 in the appendix. You do not need to
turn this in, but you will have to write queries using this new table below. </li>
<br>
    <li> Using the schema described by the diagram above and your addition, construct SQL
queries to answer the following questions. Use whatever flavor of SQL you are
comfortable with:</li>

## Answers


I have decided to use MySQL <br>
<br>
<br>
<li> <b>For each funding, return the funding_id and the agreement date.</b> </li>

        SELECT id, agreement_date
        FROM Fundings;
        
   <ul> <i>Selecting the id makes sure that we have all the entries. Selecting agreement_date completes the task.</i></ul>

<li> <b> Show me all attorneys that do NOT have a phone_number. </b></li>
        
    
        SELECT * FROM Attorney 
        WHERE phone_number IS NULL;
        
   <ul> <i>Since what information to display about the attorneys was not specified, I decided to display all of it.</i> </ul>



<li> <b>What is the total amount advanced all time? </b></li>

        SELECT SUM(advance_amount)
        FROM Fundings;
        
   <ul> <i>Since we are looking for a total advanced amount, it is appropriate to use an aggregate function.</i></ul>

<li> <b>For every case, return the attorney’s phone number and email.</b> </li>

        SELECT Attorney.phone_number, Attorney.email
        FROM Attorney
        LEFT JOIN Cases ON Attorney.id = Cases.attorney_id;
        
   <ul> <i>Since the table that contains the information we need is down the schema, a left join is appropriate. The join was performed on the id in both Attorney and Cases tables.</i></ul>

<li> <b>For every case, return the total amount funded. </b></li>
       
        SELECT Cases.id, SUM(advance_amount)
        FROM Fundings
        RIGHT JOIN Cases ON Fundings.case_id = Cases.id
        GROUP BY Fundings.case_id;
    
   <ul> <i> Since the Fundings table is upstream, it's appropriate to use the right join. GROUP BY the case_id in the Fundings table ensures that we have a set of distinct cases, and the aggregate function SUM ensures that we get the total amounts for those cases. </i></ul>

<li> <b>Return all cases that have attorney name is exactly “Sal Goodman”. </b> </li>

        SELECT *
        FROM Cases
        LEFT JOIN Cases ON Attorney.id = Cases.attorney_id
        WHERE Attorney.full_name = ‘Sal Goodman’;
        
  <ul> <i> Since I was not given which information to display about the cases, I chose to display everything. The statement WHERE ensures that the string 'Sal Goodman' is matched exactly. </i> </ul>

<li> <b>Return all case types, sorted alphabetically by name.</b></li>

        SELECT DISTINCT case_type
        FROM CaseType
        ORDER BY case_type;
        
  <ul> <i> DISTINCT ensures that every case type displayed is unique, and ORDER BY sorts case types alphabetically.</i></ul>

<li> <b>Return all case types with the name of the case type and the number of cases of that type, each in its own row, without duplicates.</b></li>

        SELECT DISTINCT case_type, COUNT(DISTINCT case_type)
        FROM CaseType;
   <ul> <i>From the table CaseType select unique case_types and COUNT those DISTINCT case_types for display.</i></ul>

<li> <b>Rank the case types by the most frequent (most number of cases) to the least frequent with the number of cases of each type.</b></li>

        SELECT DISTINCT case_type, COUNT(DISTINCT case_type)
        FROM CaseType
        ORDER BY COUNT(DISTINCT case_type) ASC;
        
  <ul><i> Same as before, but ORDER BY ... ASC makes sure that the counts of the unique case types are ordered in ascending order.</i></ul>

## Social Security Number Cleaning

When migrating client data, we often have to clean the data before inputting it into our
database. In this example, we are attempting to clean social security numbers (“SSN”s). We
have included a number of test cases your function should pass in the appendix below, but
please ensure the function meets the following conditions:
1. You should not use an existing library designed for this purpose.
2. The function should be written to clean one SSN at a time, assuming both input and output values are strings. If passed in an empty string, return an empty string.
3. The database expects that there are 11 characters in outputted SSN string in “XXX-XX-XXXX” format, where X is a integer 0-9.
4. Sometimes, input data can have only a few digits of a social security number OR too many digits.
      <ul> a. By convention, if the number of digits provided is less than 9, we add zeros in the front until they have 9 integers in the string (see Figure 2, row #1).
       b. By convention, if there are more than 9 integers provided, we assume the first 9 are the SSN (see Figure 2, row #3).</ul>
5. Hint: You do NOT have to use regular expressions.

In [1]:
'''

Import statements.

'''

import pandas as pd
from numpy import nan as Nan

In [2]:
'''

Get the dataframe with test cases.

'''

# get values
values = [
            ('9876', '000-00-9876'),
            ('987654321', '987-65-4321'),
            ('1234567891', '123-45-6789'),
            ('789526345 -- client ssn', '789-52-6345'),
            ('XXX-12-3456', '000-12-3456'),
            ('123-45-6789', '123-45-6789') # add a regular-shaped SSN 
            
]

# instantiate the dataframe
df = pd.DataFrame(values, columns = ['Input Value', 'Expected Output'])

# add the empty strings to dataframe
s2 = pd.Series([Nan,Nan], index = ['Input Value', 'Expected Output'])
df = df.append(s2, ignore_index = True)

df

,Input Value,Expected Output
0,9876,000-00-9876
1,987654321,987-65-4321
2,1234567891,123-45-6789
3,789526345 -- client ssn,789-52-6345
4,XXX-12-3456,000-12-3456
5,123-45-6789,123-45-6789
6,NaN,NaN


In [3]:
'''

Define helper functions.

'''

# split to keep words only and take only the first entry

def get_number(string):
    
    return string.split()[0] # split at whitespace and grab just the number

# replace X's with 0s

def replace_x(string):
    
        return string.replace("X", "0")
    
# if the string is convertable to a float, leave only the first 9 digits

def nine_digits(string):
    
    if string.isdigit(): # if the string is just a number, 
        
        string = string[:9] # grab the first nine digits
        
    return string

# append 0's where string is shorter than 9 digits

def get_0s(string):
    
    if len(string) < 9: # if there are not enough digits
        
        to_add = 9 - len(string) # determine how many are missing
        
        string = '0' * to_add + string # add the required number to string
    
    return string

# add dashes between the digits

def get_dashes(string):
    
    if len(string) == 9: # if the string is clean
    
        string = '-'.join([string[:3], string[3:5], string[5:]]) # add the dashes
    
    return string

In [4]:
'''

Final function to bring everything together.

'''

def clean_ssn(string):
    
    if type(string) == str: # if the string is not empty, apply helper functions.
        
        string = get_number(string)
        string = replace_x(string)
        string = nine_digits(string)
        string = get_0s(string)
        string = get_dashes(string)
        
        return string
    
    else: # if the string is empty, return empty.
        
        return string

In [5]:
'''

Test the function on the DataFrame.

'''

df['Input Value'].apply(lambda x: clean_ssn(x))

0    000-00-9876
1    987-65-4321
2    123-45-6789
3    789-52-6345
4    000-12-3456
5    123-45-6789
6            NaN
Name: Input Value, dtype: object